# Myers–Briggs Type Indicator (MBTI) Personality Prediction

## Libraries and Global Settings

In [1]:
import re
import os
import spacy
import pickle
import warnings
import numpy as np
import pandas as pd

from ipywidgets import widgets, interact
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

# python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

warnings.filterwarnings('ignore')

## Import Dataset

In [2]:
if not os.path.isfile("mbti_normalized.csv"):
    df = pd.read_csv('mbti_1.csv', encoding='utf8')
else:
    df = pd.read_csv('mbti_normalized.csv', encoding='utf8')

In [3]:
df.head(10)

,Unnamed: 0,type,posts,normalized_text
0,0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,moment sportscenter play prank life change exp...
1,1,ENTP,'I'm finding the lack of me in these posts ver...,find lack post alarming sex boring position ex...
2,2,INTP,'Good one _____ https://www.youtube.com/wat...,good course know blessing curse absolutely pos...
3,3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",dear enjoy conversation day esoteric gabbe nat...
4,4,ENTJ,'You're fired.|||That's another silly misconce...,fire silly misconception approach logically ke...
5,5,INTJ,'18/37 @.@|||Science is not perfect. No scien...,Science perfect scientist claim scientific inf...
6,6,INFJ,"'No, I can't draw on my own nails (haha). Thos...",draw nail haha professional nail yes gel mean ...
7,7,INTJ,'I tend to build up a collection of things on ...,tend build collection thing desktop use freque...
8,8,INFJ,"I'm not sure, that's a good question. The dist...",sure good question distinction dependant perce...
9,9,INTP,'https://www.youtube.com/watch?v=w8-egj0y8Qs||...,position actually let person reason unfortunat...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8675 entries, 0 to 8674
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       8675 non-null   int64 
 1   type             8675 non-null   object
 2   posts            8675 non-null   object
 3   normalized_text  8674 non-null   object
dtypes: int64(1), object(3)
memory usage: 271.2+ KB


## Data Visualization

## Preprocessing

### Text Normalization

In [5]:
def normalizer(sentence):
 
    # Remove ||| from kaggle dataset
    sentence = re.sub("[]|||[]", " ", sentence)

    # remove reddit subreddit urls
    sentence = re.sub("/r/[0-9A-Za-z]", "", sentence)

    # remove MBTI types
    MBTI_types = ['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
              'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ',
              'MBTI']
    MBTI_types = [ti.lower() for ti in MBTI_types] + [ti.lower() + 's' for ti in MBTI_types]

    tokens = nlp(sentence)

    tokens = [ti for ti in tokens if ti.lower_ not in STOP_WORDS]
    tokens = [ti for ti in tokens if not ti.is_space]
    tokens = [ti for ti in tokens if not ti.is_punct]
    tokens = [ti for ti in tokens if not ti.like_num]
    tokens = [ti for ti in tokens if not ti.like_url]
    tokens = [ti for ti in tokens if not ti.like_email]
    tokens = [ti for ti in tokens if ti.lower_ not in MBTI_types]


    # lemmatize
    tokens = [ti.lemma_ for ti in tokens if ti.lemma_ not in STOP_WORDS]
    tokens = [ti for ti in tokens if len(ti) > 1]
    
    text = " ".join(tokens)
    return text

In [6]:
if not os.path.isfile("mbti_normalized.csv"):
    df['normalized_text'] = df.posts.apply(normalizer)
    df.to_csv('mbti_normalized.csv')

In [7]:
df.head(5)

,Unnamed: 0,type,posts,normalized_text
0,0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,moment sportscenter play prank life change exp...
1,1,ENTP,'I'm finding the lack of me in these posts ver...,find lack post alarming sex boring position ex...
2,2,INTP,'Good one _____ https://www.youtube.com/wat...,good course know blessing curse absolutely pos...
3,3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",dear enjoy conversation day esoteric gabbe nat...
4,4,ENTJ,'You're fired.|||That's another silly misconce...,fire silly misconception approach logically ke...


In [8]:
# Drop unnamed columns
df = df.drop(df.columns[0], axis=1)
df = df.drop(df.columns[1], axis=1)
df

,type,normalized_text
0,INFJ,moment sportscenter play prank life change exp...
1,ENTP,find lack post alarming sex boring position ex...
2,INTP,good course know blessing curse absolutely pos...
3,INTJ,dear enjoy conversation day esoteric gabbe nat...
4,ENTJ,fire silly misconception approach logically ke...
...,...,...
8670,ISFP,IxFP think cat Fi dom reason especially websit...
8671,ENFP,thread exist someplace heck delete ooop guess ...
8672,INTP,question thing purple pill pick win lottery nu...
8673,INFP,conflicted right come want child honestly mate...


### Vectorization

In [9]:
train, test = train_test_split(df, test_size=.2)

In [10]:
train

,type,normalized_text
6522,INFP,confusion inner feeling ignore notice picture ...
3553,INTJ,leave excuse partial self promotion like know ...
6718,INFJ,time mention chance address clear mean depend ...
428,ENFP,Contra personally live debate Blair White fund...
6276,ENTP,eh type politician political argument happen a...
...,...,...
360,ENFP,hauhuhauh right Muhicz wonder step come exactl...
4189,INFJ,read comic_strip base film Professor James McA...
6519,INFP,love find think Beverly pretty typical Murray ...
3266,INFP,self confidence feel like impossible change kn...


In [11]:
X_train = train['normalized_text']
y_train = train['type']
X_test = test['normalized_text']
y_test = test['type']

In [12]:
tf_idf = TfidfVectorizer()
X_train_tf = tf_idf.fit_transform(X_train.values.astype('U'))
X_train_tf = tf_idf.transform(X_train.values.astype('U'))

print("n_samples: %d, n_features: %d" % X_train_tf.shape)

n_samples: 6940, n_features: 80779


In [13]:
X_test_tf = tf_idf.transform(X_test.values.astype('U'))

print("n_samples: %d, n_features: %d" % X_test_tf.shape)

n_samples: 1735, n_features: 80779


In [14]:
X_train = X_train_tf
X_test = X_test_tf

In [15]:
X_test.shape

(1735, 80779)

## Machine Learning

In [15]:
if not os.path.isfile("model_lr.pickle"):
    # parameter grid
    parameters_lr = {
        'penalty' : ['l1', 'l2'], 
        'C'       : np.logspace(-4, 4, 20),
        'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
        'max_iter' : [50, 100, 1000, 2500, 5000]
    }

In [18]:
if not os.path.isfile("model_lr.pickle"):
    model_LR = GridSearchCV(LogisticRegression(), param_grid = parameters_lr, scoring='accuracy', cv=5)
    model_LR = model_LR.fit(X_train, y_train)
    print("\n model_LR: best estimator across ALL searched params:\n",model_LR.best_estimator_)
    print("\n model_LR: best parameters across ALL searched params:\n",model_LR.best_params_)


 model_LR_CON: best estimator across ALL searched params:
 LogisticRegression(C=78.47599703514607, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=50, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

 model_LR_CON: best parameters across ALL searched params:
 {'C': 78.47599703514607, 'max_iter': 50, 'penalty': 'l2', 'solver': 'newton-cg'}


**After applying GridSearchCV for the Logistic Regression model,**

* The best estimator across ALL searched params = 
            LogisticRegression(C=78.47599703514607, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=50, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)
                   
* The best parameters across ALL searched params = 
            {'C': 78.47599703514607, 'max_iter': 50, 'penalty': 'l2', 'solver': 'newton-cg'}

In [16]:
model_LR = LogisticRegression(C=78.47599703514607, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=50, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)
model_LR = model_LR.fit(X_train, y_train)

In [17]:
output_LR = model_LR.predict(X_test)

In [ ]:
if not os.path.isfile("model_lr.pickle"):
    # Pickle the model
    with open('model_lr.pickle', 'wb') as files:
        pickle.dump(model_LR, files)

## Evaluation

In [18]:
# Calculatingthe accuracy of the KNN model
accuracy = accuracy_score(y_test, output_LR)
accuracy

0.4207492795389049

## Example: how to use a pickled model for predictions

```
with open('model_lr.pickle', 'rb') as f:
    pickled_model_lr = pickle.load(f)

text = 'When I was just a little girl, I keep to myself, I hardly talk to people, even when I’m in school I don’t talk to my friends in class. Most times when my teacher ask me a question, she forces words out of my mouth. My mum told me all this.'

normalized_text = normalizer(text)
vectorized_text = tf_idf.transform([normalized_text])
prediction = pickled_model_lr.predict(vectorized_text)

prediction
```

## Reference
M. J, (MBTI) Myers-Briggs Personality Type Dataset, 2017, Kaggle, May 2022. [Online]. Available: https://www.kaggle.com/datasnaek/mbti-type.